# Config agent

Chat to fill in config variables in State

In [1]:
from typing import Optional, List, Dict, Union, TypedDict, Annotated

import sqlite3

from datetime import date, datetime
from pydantic import BaseModel, Field

from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.runnables.config import RunnableConfig
from langchain_core.tools import tool

from langchain_google_vertexai import ChatVertexAI

from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode

from promptgit import Prompt

In [2]:
def add_dict(a, b):
    for key, value in b.items():
        a[key] = value

    return a

In [3]:
class ConfigState(BaseModel):
    config: Annotated[Dict[str, str], add_dict]
    name: str = None
    street: str = None
    city: str = None
    messages: Annotated[list, add_messages] = []

## Dynamicaly create tools ??

List[str, Tuple]

In [4]:
@tool
def update_name(name: str, config: RunnableConfig):
    """ 
    Update name
    """
    return {'config':{'name': name}}

@tool
def update_adress(street: str, city: str, config: RunnableConfig):
    """
    Update adress. Need both street and city
    """
    return {'config':{'street': street, 'city': city}}

tool_node = ToolNode([update_name, update_adress])

In [5]:
def initialize(state: ConfigState, config: RunnableConfig):
    if state.messages == []:
        return {'messages': [
            SystemMessage('You are setting user in the application. Your aim is to fill all the data in short conversation. Please provide name and adress (city and street)'), 
        ]}
    else:
        return

In [6]:
def chatbot(state: ConfigState, config: RunnableConfig):
    return {'messages':[llm.invoke(state.messages)]}

In [7]:
# def tool_condition():
#     pass
from langgraph.prebuilt import tools_condition

In [8]:
llm = ChatVertexAI(model='gemini-1.5-flash-002').bind_tools([update_name, update_adress])

# checkpointer = SqliteSaver(sqlite3.connect('checkpoints.sqlite', check_same_thread=False))
checkpointer = MemorySaver()

In [9]:
graph_builder = StateGraph(ConfigState)

graph_builder.add_node("initialize", initialize)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge(START, "initialize")
graph_builder.add_edge("initialize", "chatbot")
graph_builder.add_edge("chatbot", "tools")
graph_builder.add_edge("tools", END)

graph = graph_builder.compile(checkpointer=checkpointer, interrupt_before=['chatbot'])

In [10]:
config = {"configurable": {"thread_id": "42"}}

In [11]:
response = graph.invoke({'messages':[]}, config)

In [12]:
x = graph.get_state(config)

In [13]:
x.values['messages']

[SystemMessage(content='You are setting user in the application. Your aim is to fill all the data in short conversation. Please provide name and adress (city and street)', additional_kwargs={}, response_metadata={}, id='34529785-90c4-452a-a4c0-f384370e7516')]

In [14]:
graph.update_state(config, {'messages':HumanMessage('My name is Boris')})

{'configurable': {'thread_id': '42',
  'checkpoint_ns': '',
  'checkpoint_id': '1ef94b2b-65ed-65a9-8002-6034f82a31f9'}}

In [15]:
response = graph.invoke(None, config)

In [21]:
response

{'config': {},
 'messages': [SystemMessage(content='You are setting user in the application. Your aim is to fill all the data in short conversation. Please provide name and adress (city and street)', additional_kwargs={}, response_metadata={}, id='34529785-90c4-452a-a4c0-f384370e7516'),
  HumanMessage(content='My name is Boris', additional_kwargs={}, response_metadata={}, id='83f87480-e69d-4cdb-aad9-f906a6fa375c'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'update_name', 'arguments': '{"name": "Boris"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 56, 'candidates_token_count': 5, 'total_token_count': 61, 'cached_content_token_count': 0}, 'finish_reason': 'STOP'}, id='run-4606711f-ab37-4118-b2b4-13033632b0cb-0', tool_calls=[{'name': 'update_name', 'args': {'name': 'Boris'}, 'id': 'dd20cb9a-d9f7-49bc-9384-1b22870dd23e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 56, 'output_tokens': 5, 'to

In [17]:
graph.get_state(config)

StateSnapshot(values={'config': {}, 'messages': [SystemMessage(content='You are setting user in the application. Your aim is to fill all the data in short conversation. Please provide name and adress (city and street)', additional_kwargs={}, response_metadata={}, id='34529785-90c4-452a-a4c0-f384370e7516'), HumanMessage(content='My name is Boris', additional_kwargs={}, response_metadata={}, id='83f87480-e69d-4cdb-aad9-f906a6fa375c'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'update_name', 'arguments': '{"name": "Boris"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 56, 'candidates_token_count': 5, 'total_token_count': 61, 'cached_content_token_count': 0}, 'finish_reason': 'STOP'}, id='run-4606711f-ab37-4118-b2b4-13033632b0cb-0', tool_calls=[{'name': 'update_name', 'args': {'name': 'Boris'}, 'id': 'dd20cb9a-d9f7-49bc-9384-1b22870dd23e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 56, 'output